In [1]:
#!pip install chess

In [ ]:
import tkinter as tk
from tkinter import filedialog

def seleccionar_archivo():
    root = tk.Tk()
    root.withdraw()  #Oculta la ventana principal de Tkinter
    
    archivo = filedialog.askopenfilename(
        initialdir="E:/Felpipe/Escritorio",  
        title="Selecciona el archivo PGN",
        filetypes=(("Archivos PGN", "*.pgn"), ("Todos los archivos", "*.*"))
    )
    
    if archivo:
        return archivo
    else:
        return None

archivo_seleccionado = seleccionar_archivo()
print(archivo_seleccionado)

In [2]:
import os

desktop_path = "E:\\Felpipe\\Escritorio" 
file_name = "ficsgamesdb_202401_chess_movetimes_405219.pgn"

def buscar_archivo(ruta, nombre_archivo):
    for root, dirs, files in os.walk(ruta):
        if nombre_archivo in files:
            return os.path.join(root, nombre_archivo)  
    return None  

archivo_encontrado = buscar_archivo(desktop_path, file_name)

if archivo_encontrado:
    print(f"Archivo encontrado: {archivo_encontrado}")
else:
    print("Archivo NO encontrado.")

Archivo encontrado: E:\Felpipe\Escritorio\ficsgamesdb_202401_chess_movetimes_405219.pgn


In [15]:
import chess.pgn
from src.neuralcheck.logic import ChessBoard

board = ChessBoard()
file_path = archivo_encontrado

#Definir aperturas populares (primeros 3 turnos)
aperturas = {
    "Ruy López": ["e4", "e5", "Nf3", "Nc6", "Bb5"],
    "Defensa Siciliana": ["e4", "c5", "Nf3", "d6"],
    "Gambito de Dama": ["d4", "d5", "c4"],
    "Ataque Indio de Rey": ["Nf3", "d5", "g3"],
    "Apertura Inglesa": ["c4", "e5"],
    "Defensa Francesa": ["e4", "e6"],
    "Defensa Caro-Kann": ["e4", "c6"],
    "Defensa Grunfeld": ["d4", "Nf6", "c4", "g6", "Nc3", "d5"],
    "Defensa Nimzoindia": ["d4", "Nf6", "c4", "e6", "Nc3", "Bb4"]
}

resultados = {apertura:[] for apertura in aperturas.keys()}
partidas = {apertura:[] for apertura in aperturas.keys()}
tableros = {apertura:[] for apertura in aperturas.keys()}

def extraer_movimientos(partida, cabecera=False):# Función para extraer los primeros 3 turnos de una partida
    board = partida.board()
    #pdb.set_trace()
    movimientos = []
    posiciones = [board.fen()]
    for i, move in enumerate(partida.mainline_moves()):
        if i >= 6 and cabecera:  # Capturar solo los primeros 6 movimientos (3 turnos)
            break
        movimientos.append(board.san(move))  #Convertir a notación algebraica
        board.push(move)
        posiciones.append(board.fen())
    return movimientos, posiciones

def compilar_resultados(resultados, partidas, partida, movimientos, posiciones, tableros):
    apertura = partida["Apertura Detectada"]
    resultado = partida["Resultado"]
    resultado = '0-0' if resultado == '1/2-1/2' else resultado
    count01 = 0
    count10 = 0
    count00 = 0
    for elem in resultados[apertura]:
        if elem == '0-1':
            count01 += 1
        if elem == '1-0':
            count10 += 1
        if elem == '0-0':
            count00 += 1

    if (count01 < 3 and resultado == '0-1') or (count10 < 3 and resultado == '1-0') or (count00 < 3 and resultado == '0-0'):
        resultados[apertura].append(resultado)
        partidas[apertura].append(movimientos)
        tableros[apertura].append(posiciones)
        return True
    else:
        return False

def condicion_termino(resultados): #Al final esto no funciona, corto por lo sano
    condicion = True
    for apertura in resultados:
        count01 = 0
        count10 = 0
        count00 = 0
        for elem in apertura:
            if elem == '0-1':
                count01 += 1
            if elem == '1-0':
                count10 += 1
            if elem == '0-0':
                count00 += 1
        condicion = condicion and count01 == 3 and count10 == 3 and count00 == 3
    return condicion
partida_encontrada = None

with open(file_path, "r", encoding="utf-8") as pgn_file:
    k = 0
    while True:
        partida = chess.pgn.read_game(pgn_file)
        if partida is None:
            break  #Fin del archivo
        
        movs, pos = extraer_movimientos(partida, cabecera=True) #Extraigo los primeros 3 turnos (6 movimientos) como cabecera para clasificar
        
        for nombre, apertura in aperturas.items(): #Clasifico
            if movs[:len(apertura)] == apertura:
                partida_encontrada = {
                    "Apertura Detectada": nombre,
                    "Blancas": partida.headers.get("White", "Desconocido"),
                    "Negras": partida.headers.get("Black", "Desconocido"),
                    "Resultado": partida.headers.get("Result", "Desconocido"),
                    "Movimientos": movs
                }
                break  #Detengo búsqueda si encuentro una

        if partida_encontrada:
            movs, pos = extraer_movimientos(partida)
            if compilar_resultados(resultados, partidas, partida_encontrada, movs, pos, tableros):
                k += 1
                print(f"Encontrada partida {k} de {len(list(aperturas.keys())) * 9}")
                #if condicion_termino(resultados): No funca
                if k == len(list(aperturas.keys())) * 9:
                    break  

partida_encontrada

Encontrada partida 1 de 81
Encontrada partida 2 de 81
Encontrada partida 3 de 81
Encontrada partida 4 de 81
Encontrada partida 5 de 81
Encontrada partida 6 de 81
Encontrada partida 7 de 81
Encontrada partida 8 de 81
Encontrada partida 9 de 81
Encontrada partida 10 de 81
Encontrada partida 11 de 81
Encontrada partida 12 de 81
Encontrada partida 13 de 81
Encontrada partida 14 de 81
Encontrada partida 15 de 81
Encontrada partida 16 de 81
Encontrada partida 17 de 81
Encontrada partida 18 de 81
Encontrada partida 19 de 81
Encontrada partida 20 de 81
Encontrada partida 21 de 81
Encontrada partida 22 de 81
Encontrada partida 23 de 81
Encontrada partida 24 de 81
Encontrada partida 25 de 81
Encontrada partida 26 de 81
Encontrada partida 27 de 81
Encontrada partida 28 de 81
Encontrada partida 29 de 81
Encontrada partida 30 de 81
Encontrada partida 31 de 81
Encontrada partida 32 de 81
Encontrada partida 33 de 81
Encontrada partida 34 de 81
Encontrada partida 35 de 81
Encontrada partida 36 de 81
E

{'Apertura Detectada': 'Ataque Indio de Rey',
 'Blancas': 'bakkrot',
 'Negras': 'scalaQueen',
 'Resultado': '1/2-1/2',
 'Movimientos': ['Nf3', 'd5', 'g3', 'g6', 'Bg2', 'Bg7']}

In [16]:
print(f"Encontrada partida {k} de {len(list(aperturas.keys())) * 9}")

Encontrada partida 81 de 81


In [17]:
import yaml
path = 'test/test_games/'
rtypes = ['0-0', '1-0', '0-1']
saved = {
    apertura : {rtype : 0 for rtype in rtypes}
    for apertura in aperturas.keys()
}

for apertura in aperturas.keys():
    for movs, result, pos in zip(partidas[apertura], resultados[apertura], tableros[apertura]):
        ordered_movs = []
        ordered_pos = []
        for i, mov in enumerate(movs):
            if i % 2 == 0:
                ordered_movs.append([mov])
            else:
                ordered_movs[-1].append(mov)
        for i, p in enumerate(pos):
            if i % 2 == 0:
                ordered_pos.append([p])
            else:
                ordered_pos[-1].append(p)
        compiled = [[mov, p] for mov, p in zip(ordered_movs, ordered_pos)]
        
        saved[apertura][result] += 1
        filename = f'{path}{apertura} - {result} - n°{saved[apertura][result]}.yaml'        
        with open(filename, "w", encoding="utf-8") as file:
            yaml.dump(compiled, file, allow_unicode=True, default_flow_style=False)

In [26]:
import random

archivos = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
archivo_seleccionado = random.choice(archivos)
filename = os.path.join(path, archivo_seleccionado)
with open(filename, "r", encoding="utf-8") as file:
    history = yaml.safe_load(file)
print(history)

[['e4', 'c5'], ['Qf3', 'Nf6'], ['e5', 'Ng8'], ['Bc4', 'Nh6'], ['d3', 'e6'], ['Nh3', 'd5'], ['exd6', 'Qxd6'], ['Bf4', 'Qc6'], ['Qxc6+', 'Nxc6'], ['c3', 'Nf5'], ['g3', 'b6'], ['Bb5', 'Bd7'], ['Nd2', 'Rc8'], ['Nc4', 'e5'], ['Bxc6', 'Bxc6'], ['Bxe5', 'Bxh1'], ['O-O-O', 'Bf3'], ['Re1', 'f6'], ['Bd6+', 'Kf7'], ['Bf4', 'Rc6'], ['g4', 'Bxg4'], ['Ng1', 'g5'], ['Bg3', 'Rc8'], ['h3', 'Bh5'], ['Ne3', 'Re8'], ['Kd2', 'Nxg3'], ['fxg3', 'Bd6'], ['g4', 'Bg3'], ['Re2', 'Bg6'], ['Nf3', 'Re6'], ['Kc2', 'Rd8'], ['h4', 'Bxd3+'], ['Kd1', 'Bxe2+'], ['Kxe2', 'gxh4']]


In [23]:
with open(file_path, "r", encoding="utf-8") as pgn_file:
    k = 0
    while True:
        partida = chess.pgn.read_game(pgn_file)
        if partida is None:
            break
        k += 1
print(f'El archivo tiene {k} partidas')

KeyboardInterrupt: 

In [24]:
print(f'El archivo tiene {k} partidas')

El archivo tiene 50897 partidas
